In [ ]:
DID NOT USE REGISTERED FILES for functional rois?

# Longitudinal RSA Analysis: Asymmetric vs. Symmetric Visual Categories

## Overview

This notebook implements longitudinal Representational Similarity Analysis (RSA) following Liu et al. (2025) Figure 5, with key methodological improvements:

### Key Analyses:
1. **Asymmetric Categories (Face-Word):** Unilateral, ventral representations
2. **Symmetric Categories (Object-House):** Bilateral representations

### Patients:
- **TC (sub-021):** Left VOTC resection → analyze RIGHT hemisphere (3 sessions)
- **UD (sub-004):** Right VOTC resection → analyze LEFT hemisphere (5 sessions)

### Method:
- Extract all 4 categories (faces, words, objects, houses) from SAME voxels in each ROI
- Compute 4×4 representational dissimilarity matrix (RDM) per session
- Track dissimilarity trajectories across sessions
- Bootstrap null distribution (1000 iterations, shuffled labels)
- Test if regression slopes fall outside 95% CI

### Research Question:
Do asymmetric (lateralized) categories show different competitive dynamics than symmetric (bilateral) categories during cortical reorganization?

In [1]:
# Imports
import numpy as np
import pandas as pd
import nibabel as nib
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
import json
import warnings
warnings.filterwarnings('ignore')

# Plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print("✓ Libraries loaded")

✓ Libraries loaded


In [3]:
# Configuration
BASE_DIR = Path("/user_data/csimmon2/long_pt")
BETA_DIR = BASE_DIR / "analyses" / "beta_extraction"
OUTPUT_DIR = BASE_DIR / "analyses" / "longitudinal_rsa_final"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Patient configurations
PATIENTS = {
    'sub-004': {
        'code': 'UD',
        'sessions': ['01', '02', '03', '05', '06'],
        'hemi': 'l',  # Analyze left (right was resected)
        'resection': 'right',
        'expected': 'Increasing face-word dissimilarity (competition)'
    },
    'sub-021': {
        'code': 'TC',
        'sessions': ['01', '02', '03'],
        'hemi': 'r',  # Analyze right (left was resected)
        'resection': 'left',
        'expected': 'Increasing face-word dissimilarity (competition)'
    }
}

# ROI configurations
ROI_ANALYSES = {
    'face_word': {
        'name': 'face_word_dual_cluster',
        'description': 'Asymmetric/Unilateral Categories',
        'dissim_pair': ('faces', 'words'),
        'color': '#FF1493'  # Deep pink
    },
    'object_house': {
        'name': 'object_house_dual_cluster',
        'description': 'Symmetric/Bilateral Categories',
        'dissim_pair': ('houses', 'objects'),
        'color': '#00CED1'  # Dark turquoise
    }
}

N_BOOTSTRAP = 1000
RANDOM_SEED = 42

print(f"Base directory: {BASE_DIR}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Patients: {[p['code'] for p in PATIENTS.values()]}")
print(f"Bootstrap iterations: {N_BOOTSTRAP}")

Base directory: /user_data/csimmon2/long_pt
Output directory: /user_data/csimmon2/long_pt/analyses/longitudinal_rsa_final
Patients: ['UD', 'TC']
Bootstrap iterations: 1000


In [ ]:
# CELL: Beta Extraction Configuration
# ============================================================================

# Cope mappings - confirmed from design
COPE_MAP = {
    'faces': 6,    # Face > all others
    'houses': 7,   # House > all others
    'objects': 8,  # Object > all others
    'words': 9     # Word > all others
}

# ROI configurations (matching your existing ROI_ANALYSES dict)
ROI_ANALYSES = {
    'face_word': {'name': 'face_word_dual_cluster'},
    'object_house': {'name': 'object_house_dual_cluster'}
}

In [34]:
# CELL: Beta Extraction Functions
# ============================================================================

def extract_betas_from_roi(cope_file, roi_file):
    """Extract mean beta values from ROI voxels"""
    cope_img = nib.load(cope_file)
    roi_img = nib.load(roi_file)
    
    cope_data = cope_img.get_fdata()
    roi_data = roi_img.get_fdata()
    
    # Get voxels in ROI (non-zero)
    roi_mask = roi_data > 0
    
    if not roi_mask.any():
        return None
    
    # Extract betas from ROI voxels
    betas = cope_data[roi_mask]
    
    return betas

def extract_session_betas(subject_id, session, roi_configs, base_dir):
    """Extract betas using ses-01 registered copes"""
    
    hemi = PATIENTS[subject_id]['hemi']
    roi_dir = base_dir / subject_id / 'ses-01' / 'ROIs'
    session_dir = base_dir / subject_id / f'ses-{session}'
    feat_dir = session_dir / 'derivatives' / 'fsl' / 'loc' / 'HighLevel.gfeat'
    
    results = {}
    
    for roi_type, roi_config in roi_configs.items():
        roi_name = f"{hemi}_{roi_config['name']}.nii.gz"
        roi_file = roi_dir / roi_name
        
        if not roi_file.exists():
            print(f"  ✗ ROI not found: {roi_file}")
            continue
        
        roi_betas = {}
        
        for cat_name, cope_num in COPE_MAP.items():
            cope_dir = feat_dir / f'cope{cope_num}.feat' / 'stats'
            
            # Use ses-01 registered files for non-ses-01 sessions
            if session == '01':
                cope_file = cope_dir / 'cope1.nii.gz'
            else:
                cope_file = cope_dir / 'cope1_ses01.nii.gz'
            
            if not cope_file.exists():
                print(f"  ✗ Cope not found: {cope_file}")
                continue
            
            betas = extract_betas_from_roi(cope_file, roi_file)
            
            if betas is not None:
                roi_betas[cat_name] = betas
        
        if len(roi_betas) == 4:
            results[roi_type] = roi_betas
            print(f"  ✓ {roi_type}: {len(roi_betas['faces'])} voxels")
    
    return results if len(results) == 2 else None

In [35]:
# CELL: Extract Betas for All Sessions
print("\n" + "="*80)
print("BETA EXTRACTION")
print("="*80)

all_session_data = {}

for subject_id, config in PATIENTS.items():
    code = config['code']
    print(f"\n{code} ({subject_id}):")
    print("-" * 60)
    
    subject_data = {}
    
    for session in config['sessions']:
        print(f"\nSession {session}:")
        
        betas = extract_session_betas(subject_id, session, ROI_ANALYSES, BASE_DIR)
        
        if betas is not None:
            subject_data[session] = betas
            print(f"  ✓ Extraction complete")
        else:
            print(f"  ✗ Extraction failed")
    
    if len(subject_data) > 0:
        all_session_data[code] = subject_data
        print(f"\n✓ {code}: {len(subject_data)} sessions extracted")
    else:
        print(f"\n✗ {code}: No sessions extracted")

print("\n" + "="*80)
print(f"Total patients with data: {len(all_session_data)}")
print("="*80)


BETA EXTRACTION

UD (sub-004):
------------------------------------------------------------

Session 01:
  ✓ face_word: 8023 voxels
  ✓ object_house: 13734 voxels
  ✓ Extraction complete

Session 02:
  ✗ Cope not found: /user_data/csimmon2/long_pt/sub-004/ses-02/derivatives/fsl/loc/HighLevel.gfeat/cope6.feat/stats/cope1_ses01.nii.gz
  ✗ Cope not found: /user_data/csimmon2/long_pt/sub-004/ses-02/derivatives/fsl/loc/HighLevel.gfeat/cope7.feat/stats/cope1_ses01.nii.gz
  ✗ Cope not found: /user_data/csimmon2/long_pt/sub-004/ses-02/derivatives/fsl/loc/HighLevel.gfeat/cope8.feat/stats/cope1_ses01.nii.gz
  ✗ Cope not found: /user_data/csimmon2/long_pt/sub-004/ses-02/derivatives/fsl/loc/HighLevel.gfeat/cope9.feat/stats/cope1_ses01.nii.gz
  ✗ Cope not found: /user_data/csimmon2/long_pt/sub-004/ses-02/derivatives/fsl/loc/HighLevel.gfeat/cope6.feat/stats/cope1_ses01.nii.gz
  ✗ Cope not found: /user_data/csimmon2/long_pt/sub-004/ses-02/derivatives/fsl/loc/HighLevel.gfeat/cope7.feat/stats/cope1_se

In [12]:
# CELL: Compute RDMs from Extracted Betas
# ============================================================================

def compute_rdm_from_betas(beta_dict):
    """Compute 4x4 RDM from beta dictionary"""
    categories = ['faces', 'words', 'objects', 'houses']
    
    # Stack into matrix
    beta_matrix = np.vstack([beta_dict[cat] for cat in categories])
    
    # Correlate across voxels
    corr_matrix = np.corrcoef(beta_matrix)
    
    # Convert to dissimilarity
    rdm = 1 - corr_matrix
    
    return rdm, categories


print("\n" + "="*80)
print("COMPUTING RDMs")
print("="*80)

# Store RDMs and dissimilarities
rsa_data = {}

for code, sessions_dict in all_session_data.items():
    print(f"\n{code}:")
    print("-" * 60)
    
    rsa_data[code] = {
        'face_word': [],
        'object_house': []
    }
    
    for session, beta_data in sorted(sessions_dict.items()):
        print(f"\nSession {session}:")
        
        # Face-Word ROI
        if 'face_word' in beta_data:
            rdm_fw, cats = compute_rdm_from_betas(beta_data['face_word'])
            face_idx = cats.index('faces')
            word_idx = cats.index('words')
            dissim_fw = rdm_fw[face_idx, word_idx]
            
            rsa_data[code]['face_word'].append({
                'session': int(session),
                'rdm': rdm_fw,
                'dissimilarity': dissim_fw,
                'n_voxels': len(beta_data['face_word']['faces'])
            })
            print(f"  Face-Word: dissim={dissim_fw:.4f}")
        
        # Object-House ROI  
        if 'object_house' in beta_data:
            rdm_oh, cats = compute_rdm_from_betas(beta_data['object_house'])
            house_idx = cats.index('houses')
            object_idx = cats.index('objects')
            dissim_oh = rdm_oh[house_idx, object_idx]
            
            rsa_data[code]['object_house'].append({
                'session': int(session),
                'rdm': rdm_oh,
                'dissimilarity': dissim_oh,
                'n_voxels': len(beta_data['object_house']['faces'])
            })
            print(f"  House-Object: dissim={dissim_oh:.4f}")

print("\n" + "="*80)
print("RDM COMPUTATION COMPLETE")
print("="*80)


COMPUTING RDMs

UD:
------------------------------------------------------------

Session 01:
  Face-Word: dissim=1.4953
  House-Object: dissim=1.6898

Session 02:
  Face-Word: dissim=0.4737
  House-Object: dissim=1.1656

Session 03:
  Face-Word: dissim=0.9712
  House-Object: dissim=0.8472

Session 05:
  Face-Word: dissim=1.1347
  House-Object: dissim=1.1203

Session 06:
  Face-Word: dissim=0.6663
  House-Object: dissim=0.9814

TC:
------------------------------------------------------------

Session 01:
  Face-Word: dissim=1.7938
  House-Object: dissim=1.8621

Session 02:
  Face-Word: dissim=0.9567
  House-Object: dissim=1.4962

Session 03:
  Face-Word: dissim=1.2059
  House-Object: dissim=1.5631

RDM COMPUTATION COMPLETE


## Load Session Inventory

In [14]:
# Check what we extracted
print("\n" + "="*80)
print("EXTRACTED DATA SUMMARY")
print("="*80)

for code, roi_dict in rsa_data.items():
    print(f"\n{code}:")
    print(f"  Face-Word: {len(roi_dict['face_word'])} sessions")
    print(f"  Object-House: {len(roi_dict['object_house'])} sessions")


EXTRACTED DATA SUMMARY

UD:
  Face-Word: 5 sessions
  Object-House: 5 sessions

TC:
  Face-Word: 3 sessions
  Object-House: 3 sessions


## Core Functions

In [15]:
def load_session_data(session_id, beta_dir):
    """Load beta matrix, ROI info, and metadata"""
    session_dir = beta_dir / session_id
    
    if not session_dir.exists():
        return None, None, None
    
    beta_file = session_dir / "beta_matrix.npy"
    roi_file = session_dir / "roi_info.csv"
    metadata_file = session_dir / "metadata.json"
    
    if not all([beta_file.exists(), roi_file.exists(), metadata_file.exists()]):
        return None, None, None
    
    beta_matrix = np.load(beta_file)
    roi_info = pd.read_csv(roi_file)
    
    with open(metadata_file, 'r') as f:
        metadata = json.load(f)
    
    return beta_matrix, roi_info, metadata


def extract_category_betas_from_roi(beta_matrix, roi_info, condition_order, 
                                    roi_name, hemisphere):
    """Extract betas for all 4 categories from specified ROI
    
    Returns:
    --------
    category_betas : dict
        {category: beta_array} for faces, words, objects, houses
    n_voxels : int
    """
    # Select voxels from this ROI in this hemisphere
    hemi_roi_name = f"{hemisphere}_{roi_name}"
    voxel_mask = roi_info['roi_type'].str.contains(roi_name, case=False, na=False)
    voxel_mask &= roi_info['roi_type'].str.startswith(f"{hemisphere}_", na=False)
    
    n_selected = voxel_mask.sum()
    
    if n_selected == 0:
        return None, 0
    
    # Extract all 4 categories from same voxels
    categories = ['faces', 'words', 'objects', 'houses']
    category_betas = {}
    
    for cat in categories:
        if cat not in condition_order:
            return None, 0
        cat_idx = condition_order.index(cat)
        category_betas[cat] = beta_matrix[cat_idx, voxel_mask]
    
    return category_betas, n_selected


def compute_rdm(category_betas):
    """Compute 4×4 representational dissimilarity matrix
    
    RDM[i,j] = 1 - correlation(category_i, category_j)
    """
    categories = ['faces', 'words', 'objects', 'houses']
    beta_matrix = np.vstack([category_betas[cat] for cat in categories])
    correlation_matrix = np.corrcoef(beta_matrix)
    rdm = 1 - correlation_matrix
    return rdm, categories


def extract_dissimilarity(rdm, categories, cat1, cat2):
    """Extract specific dissimilarity pair from RDM"""
    idx1 = categories.index(cat1)
    idx2 = categories.index(cat2)
    return rdm[idx1, idx2]


print("✓ Core functions defined")

✓ Core functions defined


## Process Patient Trajectories

In [16]:
def process_patient_rsa(subject_id, patient_config, roi_config, 
                       session_inventory, beta_dir):
    """Process one ROI analysis for one patient
    
    Returns:
    --------
    trajectory_df : pd.DataFrame with columns:
        - session, session_id, n_voxels, dissimilarity, rdm
    """
    code = patient_config['code']
    hemisphere = patient_config['hemi']
    roi_name = roi_config['name']
    cat1, cat2 = roi_config['dissim_pair']
    
    print(f"\n{'='*70}")
    print(f"{code} - {roi_config['description']}")
    print(f"{'='*70}")
    print(f"ROI: {hemisphere}_{roi_name}")
    print(f"Tracking: {cat1}-{cat2} dissimilarity\n")
    
    # Get sessions
    patient_sessions = session_inventory[
        session_inventory['subject'] == subject_id
    ].sort_values('session')
    
    trajectory = []
    
    for _, session_row in patient_sessions.iterrows():
        session_id = session_row['session_id']
        session_num = session_row['session']
        
        print(f"Session {session_num}:", end=' ')
        
        # Load data
        beta_matrix, roi_info, metadata = load_session_data(session_id, beta_dir)
        if beta_matrix is None:
            print("✗ Load failed")
            continue
        
        # Extract betas
        category_betas, n_voxels = extract_category_betas_from_roi(
            beta_matrix, roi_info, metadata['condition_order'],
            roi_name, hemisphere
        )
        
        if category_betas is None:
            print("✗ No voxels")
            continue
        
        # Compute RDM
        rdm, categories = compute_rdm(category_betas)
        
        # Extract dissimilarity
        dissim = extract_dissimilarity(rdm, categories, cat1, cat2)
        
        print(f"✓ {n_voxels} voxels, dissim={dissim:.4f}")
        
        trajectory.append({
            'subject': subject_id,
            'code': code,
            'session': session_num,
            'session_id': session_id,
            'hemisphere': hemisphere,
            'roi_type': roi_config['name'],
            'n_voxels': n_voxels,
            'dissimilarity': dissim,
            'rdm': rdm,
            'categories': categories,
            'pair': f"{cat1}-{cat2}"
        })
    
    if len(trajectory) == 0:
        print(f"\n✗ No valid sessions")
        return None
    
    trajectory_df = pd.DataFrame(trajectory)
    print(f"\n✓ Extracted {len(trajectory)} sessions")
    
    return trajectory_df

print("✓ Patient processing function defined")

✓ Patient processing function defined


## Bootstrap Null Distribution

In [31]:
# ============================================================================
# CELL: Bootstrap and Statistical Testing
# ============================================================================

def bootstrap_null_slopes(trajectory_list, cat1_name, cat2_name, n_bootstrap=1000, seed=42):
    """Generate null distribution from shuffled RDMs"""
    np.random.seed(seed)
    
    sessions = np.array([t['session'] for t in trajectory_list])
    n_sessions = len(sessions)
    categories = ['faces', 'words', 'objects', 'houses']
    cat1_idx = categories.index(cat1_name)
    cat2_idx = categories.index(cat2_name)
    
    print(f"  Bootstrapping ({n_bootstrap} iterations)...", end=' ')
    
    null_slopes = []
    for i in range(n_bootstrap):
        shuffled_dissims = []
        for t in trajectory_list:
            rdm = t['rdm'].copy()
            shuffle_idx = np.random.permutation(4)
            rdm_shuffled = rdm[shuffle_idx, :][:, shuffle_idx]
            shuffled_dissims.append(rdm_shuffled[cat1_idx, cat2_idx])
        
        slope = np.polyfit(sessions, shuffled_dissims, 1)[0] if n_sessions >= 2 else 0
        null_slopes.append(slope)
    
    print("✓")
    return np.array(null_slopes)


def test_significance(trajectory_list, null_slopes):
    """Test observed slope against null"""
    sessions = np.array([t['session'] for t in trajectory_list])
    dissims = np.array([t['dissimilarity'] for t in trajectory_list])
    
    slope_obs = np.polyfit(sessions, dissims, 1)[0]
    ci_lower, ci_upper = np.percentile(null_slopes, [2.5, 97.5])
    p_value = np.mean(np.abs(null_slopes) >= np.abs(slope_obs))
    significant = (slope_obs < ci_lower) or (slope_obs > ci_upper)
    
    return {
        'slope_obs': slope_obs,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper,
        'p_value': p_value,
        'significant': significant,
        'null_slopes': null_slopes
    }


# Run analysis
all_results = {}

for code, roi_dict in rsa_data.items():
    print(f"\n{'='*70}")
    print(f"{code} - STATISTICAL TESTING")
    print('='*70)
    
    results = {}
    
    # Face-Word
    if len(roi_dict['face_word']) >= 2:
        print("\nFace-Word:")
        null_fw = bootstrap_null_slopes(roi_dict['face_word'], 'faces', 'words', N_BOOTSTRAP, RANDOM_SEED)
        stats_fw = test_significance(roi_dict['face_word'], null_fw)
        results['face_word'] = {'trajectory': roi_dict['face_word'], 'stats': stats_fw}
        print(f"  Slope: {stats_fw['slope_obs']:.4f} (p={stats_fw['p_value']:.3f}) ({'***' if stats_fw['significant'] else 'ns'})")
    
    # Object-House
    if len(roi_dict['object_house']) >= 2:
        print("\nObject-House:")
        null_oh = bootstrap_null_slopes(roi_dict['object_house'], 'houses', 'objects', N_BOOTSTRAP, RANDOM_SEED)
        stats_oh = test_significance(roi_dict['object_house'], null_oh)
        results['object_house'] = {'trajectory': roi_dict['object_house'], 'stats': stats_oh}
        print(f"  Slope: {stats_oh['slope_obs']:.4f} (p={stats_oh['p_value']:.3f}) ({'***' if stats_oh['significant'] else 'ns'})")
    
    all_results[code] = results

print("\n" + "="*70)
print("ANALYSIS COMPLETE")
print("="*70)


UD - STATISTICAL TESTING

Face-Word:
  Bootstrapping (1000 iterations)... ✓
  Slope: -0.0635 (p=0.507) (ns)

Object-House:
  Bootstrapping (1000 iterations)... ✓
  Slope: -0.0978 (p=0.108) (ns)

TC - STATISTICAL TESTING

Face-Word:
  Bootstrapping (1000 iterations)... ✓
  Slope: -0.2940 (p=0.541) (ns)

Object-House:
  Bootstrapping (1000 iterations)... ✓
  Slope: -0.1495 (p=0.661) (ns)

ANALYSIS COMPLETE


## Statistical Testing

In [26]:
def test_slope_significance(trajectory_df, null_slopes):
    """Test if observed slope falls outside 95% CI of null distribution"""
    
    sessions = trajectory_df['session'].values
    dissims = trajectory_df['dissimilarity'].values
    
    # Observed slope
    slope_obs = np.polyfit(sessions, dissims, 1)[0]
    
    # 95% CI
    ci_lower, ci_upper = np.percentile(null_slopes, [2.5, 97.5])
    
    # Two-tailed p-value
    p_value = np.mean(np.abs(null_slopes) >= np.abs(slope_obs))
    
    # Significance
    significant = (slope_obs < ci_lower) or (slope_obs > ci_upper)
    
    return {
        'slope_obs': slope_obs,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper,
        'p_value': p_value,
        'significant': significant,
        'null_slopes': null_slopes
    }

print("✓ Statistical testing function defined")

✓ Statistical testing function defined


## Visualization Functions

In [27]:
def plot_patient_complete_analysis(trajectory_fw, trajectory_oh, 
                                   stats_fw, stats_oh, 
                                   patient_config, output_dir):
    """Create comprehensive 2×3 panel figure
    
    Top row: Face-Word (asymmetric)
    Bottom row: Object-House (symmetric)
    
    Columns:
    1. Trajectory
    2. Example RDMs (first & last session)
    3. Bootstrap histogram
    """
    code = patient_config['code']
    
    fig = plt.figure(figsize=(20, 12))
    gs = fig.add_gridspec(2, 3, hspace=0.3, wspace=0.3)
    
    # ===== TOP ROW: FACE-WORD =====
    
    # Panel A: Trajectory
    ax1 = fig.add_subplot(gs[0, 0])
    sessions_fw = trajectory_fw['session'].values
    dissims_fw = trajectory_fw['dissimilarity'].values
    
    ax1.plot(sessions_fw, dissims_fw, 'o-', color='#FF1493', 
            linewidth=4, markersize=14, markeredgecolor='black', markeredgewidth=2)
    ax1.set_xlabel('Session', fontsize=14, fontweight='bold')
    ax1.set_ylabel('Dissimilarity (1-r)', fontsize=14, fontweight='bold')
    ax1.set_title(f'{code} - Face-Word\n(Asymmetric/Unilateral)', 
                 fontsize=16, fontweight='bold')
    ax1.grid(alpha=0.3)
    
    # Add slope annotation
    slope_text = f"Slope: {stats_fw['slope_obs']:.4f}"
    if stats_fw['significant']:
        slope_text += " ***"
    ax1.text(0.05, 0.95, slope_text, transform=ax1.transAxes,
            fontsize=12, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # Panel B: RDMs
    ax2 = fig.add_subplot(gs[0, 1])
    rdm_first = trajectory_fw.iloc[0]['rdm']
    rdm_last = trajectory_fw.iloc[-1]['rdm']
    categories = trajectory_fw.iloc[0]['categories']
    
    # Show side-by-side RDMs
    rdm_compare = np.hstack([rdm_first, rdm_last])
    im = ax2.imshow(rdm_compare, cmap='viridis', vmin=0, vmax=2)
    ax2.set_title(f'RDMs: Session {sessions_fw[0]} → {sessions_fw[-1]}',
                 fontsize=14, fontweight='bold')
    ax2.set_yticks(range(4))
    ax2.set_yticklabels(categories, fontsize=10)
    ax2.set_xticks([])
    plt.colorbar(im, ax=ax2, fraction=0.046, pad=0.04)
    
    # Panel C: Bootstrap
    ax3 = fig.add_subplot(gs[0, 2])
    ax3.hist(stats_fw['null_slopes'], bins=50, color='gold', alpha=0.7, 
            edgecolor='black', linewidth=1)
    ax3.axvline(stats_fw['slope_obs'], color='#FF1493', linewidth=4,
               label=f"Observed (p={stats_fw['p_value']:.3f})")
    ax3.axvline(stats_fw['ci_lower'], color='gray', linestyle='--', linewidth=2)
    ax3.axvline(stats_fw['ci_upper'], color='gray', linestyle='--', linewidth=2,
               label='95% CI')
    ax3.set_xlabel('Regression Slope', fontsize=14, fontweight='bold')
    ax3.set_ylabel('Frequency', fontsize=14, fontweight='bold')
    ax3.set_title('Bootstrap Test', fontsize=14, fontweight='bold')
    ax3.legend(fontsize=10)
    ax3.grid(alpha=0.3, axis='y')
    
    # ===== BOTTOM ROW: OBJECT-HOUSE =====
    
    # Panel D: Trajectory
    ax4 = fig.add_subplot(gs[1, 0])
    sessions_oh = trajectory_oh['session'].values
    dissims_oh = trajectory_oh['dissimilarity'].values
    
    ax4.plot(sessions_oh, dissims_oh, 's-', color='#00CED1',
            linewidth=4, markersize=14, markeredgecolor='black', markeredgewidth=2)
    ax4.set_xlabel('Session', fontsize=14, fontweight='bold')
    ax4.set_ylabel('Dissimilarity (1-r)', fontsize=14, fontweight='bold')
    ax4.set_title(f'{code} - House-Object\n(Symmetric/Bilateral)',
                 fontsize=16, fontweight='bold')
    ax4.grid(alpha=0.3)
    
    # Add slope annotation
    slope_text = f"Slope: {stats_oh['slope_obs']:.4f}"
    if stats_oh['significant']:
        slope_text += " ***"
    ax4.text(0.05, 0.95, slope_text, transform=ax4.transAxes,
            fontsize=12, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.5))
    
    # Panel E: RDMs
    ax5 = fig.add_subplot(gs[1, 1])
    rdm_first = trajectory_oh.iloc[0]['rdm']
    rdm_last = trajectory_oh.iloc[-1]['rdm']
    
    rdm_compare = np.hstack([rdm_first, rdm_last])
    im = ax5.imshow(rdm_compare, cmap='viridis', vmin=0, vmax=2)
    ax5.set_title(f'RDMs: Session {sessions_oh[0]} → {sessions_oh[-1]}',
                 fontsize=14, fontweight='bold')
    ax5.set_yticks(range(4))
    ax5.set_yticklabels(categories, fontsize=10)
    ax5.set_xticks([])
    plt.colorbar(im, ax=ax5, fraction=0.046, pad=0.04)
    
    # Panel F: Bootstrap
    ax6 = fig.add_subplot(gs[1, 2])
    ax6.hist(stats_oh['null_slopes'], bins=50, color='gold', alpha=0.7,
            edgecolor='black', linewidth=1)
    ax6.axvline(stats_oh['slope_obs'], color='#00CED1', linewidth=4,
               label=f"Observed (p={stats_oh['p_value']:.3f})")
    ax6.axvline(stats_oh['ci_lower'], color='gray', linestyle='--', linewidth=2)
    ax6.axvline(stats_oh['ci_upper'], color='gray', linestyle='--', linewidth=2,
               label='95% CI')
    ax6.set_xlabel('Regression Slope', fontsize=14, fontweight='bold')
    ax6.set_ylabel('Frequency', fontsize=14, fontweight='bold')
    ax6.set_title('Bootstrap Test', fontsize=14, fontweight='bold')
    ax6.legend(fontsize=10)
    ax6.grid(alpha=0.3, axis='y')
    
    plt.suptitle(f'{code} - Longitudinal RSA: Asymmetric vs. Symmetric Categories',
                fontsize=18, fontweight='bold', y=0.995)
    
    # Save
    output_file = output_dir / f'{code}_complete_rsa_analysis.png'
    plt.savefig(output_file, dpi=300, bbox_inches='tight', facecolor='white')
    print(f"\n✓ Figure saved: {output_file}")
    
    plt.show()
    
    return fig

print("✓ Visualization functions defined")

✓ Visualization functions defined


## Main Analysis Pipeline

In [28]:
# ============================================================================
# CELL: Bootstrap and Statistical Testing
# ============================================================================

def bootstrap_null_slopes(trajectory_list, n_bootstrap=1000, seed=42):
    """Generate null distribution from shuffled RDMs"""
    np.random.seed(seed)
    
    sessions = np.array([t['session'] for t in trajectory_list])
    n_sessions = len(sessions)
    
    print(f"  Bootstrapping ({n_bootstrap} iterations)...", end=' ')
    
    null_slopes = []
    for i in range(n_bootstrap):
        shuffled_dissims = []
        for t in trajectory_list:
            rdm = t['rdm'].copy()
            shuffle_idx = np.random.permutation(4)
            rdm_shuffled = rdm[shuffle_idx, :][:, shuffle_idx]
            shuffled_dissims.append(rdm_shuffled[0, 1])  # Any pair
        
        slope = np.polyfit(sessions, shuffled_dissims, 1)[0] if n_sessions >= 2 else 0
        null_slopes.append(slope)
    
    print("✓")
    return np.array(null_slopes)


def test_significance(trajectory_list, null_slopes):
    """Test observed slope against null"""
    sessions = np.array([t['session'] for t in trajectory_list])
    dissims = np.array([t['dissimilarity'] for t in trajectory_list])
    
    slope_obs = np.polyfit(sessions, dissims, 1)[0]
    ci_lower, ci_upper = np.percentile(null_slopes, [2.5, 97.5])
    p_value = np.mean(np.abs(null_slopes) >= np.abs(slope_obs))
    significant = (slope_obs < ci_lower) or (slope_obs > ci_upper)
    
    return {
        'slope_obs': slope_obs,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper,
        'p_value': p_value,
        'significant': significant,
        'null_slopes': null_slopes
    }


# Run analysis
all_results = {}

for code, roi_dict in rsa_data.items():
    print(f"\n{'='*70}")
    print(f"{code} - STATISTICAL TESTING")
    print('='*70)
    
    results = {}
    
    # Face-Word
    if len(roi_dict['face_word']) >= 2:
        print("\nFace-Word:")
        null_fw = bootstrap_null_slopes(roi_dict['face_word'], N_BOOTSTRAP, RANDOM_SEED)
        stats_fw = test_significance(roi_dict['face_word'], null_fw)
        results['face_word'] = {'trajectory': roi_dict['face_word'], 'stats': stats_fw}
        print(f"  Slope: {stats_fw['slope_obs']:.4f} ({'SIG' if stats_fw['significant'] else 'ns'})")
    
    # Object-House
    if len(roi_dict['object_house']) >= 2:
        print("\nObject-House:")
        null_oh = bootstrap_null_slopes(roi_dict['object_house'], N_BOOTSTRAP, RANDOM_SEED)
        stats_oh = test_significance(roi_dict['object_house'], null_oh)
        results['object_house'] = {'trajectory': roi_dict['object_house'], 'stats': stats_oh}
        print(f"  Slope: {stats_oh['slope_obs']:.4f} ({'SIG' if stats_oh['significant'] else 'ns'})")
    
    all_results[code] = results

print("\n" + "="*70)
print("ANALYSIS COMPLETE")
print("="*70)


UD - STATISTICAL TESTING

Face-Word:
  Bootstrapping (1000 iterations)... ✓
  Slope: -0.0635 (ns)

Object-House:
  Bootstrapping (1000 iterations)... ✓
  Slope: -0.0978 (ns)

TC - STATISTICAL TESTING

Face-Word:
  Bootstrapping (1000 iterations)... ✓
  Slope: -0.2940 (ns)

Object-House:
  Bootstrapping (1000 iterations)... ✓
  Slope: -0.1495 (ns)

ANALYSIS COMPLETE


## Summary Table

In [30]:
# Create comprehensive summary
summary_data = []

for code, results in all_results.items():
    row = {'Patient': code}
    
    if 'face_word' in results:
        stats_fw = results['face_word']['stats']
        n_sess = len(results['face_word']['trajectory'])
        row.update({
            'N_Sessions': n_sess,
            'FW_Slope': f"{stats_fw['slope_obs']:.4f}",
            'FW_p': f"{stats_fw['p_value']:.3f}",
            'FW_Sig': '***' if stats_fw['significant'] else 'ns'
        })
    
    if 'object_house' in results:
        stats_oh = results['object_house']['stats']
        row.update({
            'OH_Slope': f"{stats_oh['slope_obs']:.4f}",
            'OH_p': f"{stats_oh['p_value']:.3f}",
            'OH_Sig': '***' if stats_oh['significant'] else 'ns'
        })
    
    summary_data.append(row)

summary_df = pd.DataFrame(summary_data)

print("\n" + "="*80)
print("SUMMARY TABLE")
print("="*80)
print(summary_df.to_string(index=False))

# Save
summary_file = OUTPUT_DIR / 'longitudinal_rsa_summary.csv'
summary_df.to_csv(summary_file, index=False)
print(f"\n✓ Summary saved: {summary_file}")


SUMMARY TABLE
Patient  N_Sessions FW_Slope  FW_p FW_Sig OH_Slope  OH_p OH_Sig
     UD           5  -0.0635 0.507     ns  -0.0978 0.112     ns
     TC           3  -0.2940 0.541     ns  -0.1495 0.644     ns

✓ Summary saved: /user_data/csimmon2/long_pt/analyses/longitudinal_rsa_final/longitudinal_rsa_summary.csv


## Interpretation Guide

### Expected Patterns:

**Asymmetric Categories (Face-Word):**
- **TC & UD:** Should show INCREASING dissimilarity (positive slope, significant)
- Interpretation: Competitive differentiation as representations reorganize in single hemisphere
- Higher initial overlap → must differentiate over time

**Symmetric Categories (Object-House):**
- **TC & UD:** Should show STABLE dissimilarity (slope near zero, not significant)
- Interpretation: Bilateral categories less affected by unilateral resection
- Already differentiated in typical development

### Key Comparisons:

1. **Within-patient:** Face-Word slope > House-Object slope
2. **Cross-patient:** Both TC and UD show similar patterns
3. **Mechanism:** Pattern differentiation (correlation-based), not magnitude changes

### Liu et al. (2025) Reference Values:
- **TC Face-Word:** 0.15 (significant)
- **UD Face-Word:** 0.12 (significant, first 4 sessions)
- **TC House-Object:** 0.01 (not significant)
- **UD House-Object:** 0.02 (not significant)

In [23]:
# Check image spaces match
subject_id = 'sub-004'
session = '01'

roi_file = BASE_DIR / subject_id / 'ses-01' / 'ROIs' / 'l_face_word_dual_cluster.nii.gz'
cope_file = BASE_DIR / subject_id / f'ses-{session}' / 'derivatives' / 'fsl' / 'loc' / 'HighLevel.gfeat' / 'cope6.feat' / 'stats' / 'cope1.nii.gz'

roi_img = nib.load(roi_file)
cope_img = nib.load(cope_file)

print("ROI shape:", roi_img.shape)
print("Cope shape:", cope_img.shape)
print("\nROI affine:\n", roi_img.affine)
print("\nCope affine:\n", cope_img.affine)
print("\nShapes match:", roi_img.shape == cope_img.shape)
print("Affines match:", np.allclose(roi_img.affine, cope_img.affine))

ROI shape: (176, 256, 256)
Cope shape: (176, 256, 256)

ROI affine:
 [[   1.     0.     0.   -83.5]
 [   0.     1.     0.  -127. ]
 [   0.     0.     1.  -127. ]
 [   0.     0.     0.     1. ]]

Cope affine:
 [[   1.     0.     0.   -83.5]
 [   0.     1.     0.  -127. ]
 [   0.     0.     1.  -127. ]
 [   0.     0.     0.     1. ]]

Shapes match: True
Affines match: True


In [32]:
# ============================================================================
# CELL: Diagnostic - View Dissimilarity Trajectories
# ============================================================================

for code, roi_dict in rsa_data.items():
    print(f"\n{code}:")
    print("-" * 60)
    
    print("\nFace-Word dissimilarity trajectory:")
    for t in roi_dict['face_word']:
        print(f"  Session {t['session']:02d}: {t['dissimilarity']:.4f}")
    
    print("\nObject-House dissimilarity trajectory:")
    for t in roi_dict['object_house']:
        print(f"  Session {t['session']:02d}: {t['dissimilarity']:.4f}")


UD:
------------------------------------------------------------

Face-Word dissimilarity trajectory:
  Session 01: 1.4953
  Session 02: 0.4737
  Session 03: 0.9712
  Session 05: 1.1347
  Session 06: 0.6663

Object-House dissimilarity trajectory:
  Session 01: 1.6898
  Session 02: 1.1656
  Session 03: 0.8472
  Session 05: 1.1203
  Session 06: 0.9814

TC:
------------------------------------------------------------

Face-Word dissimilarity trajectory:
  Session 01: 1.7938
  Session 02: 0.9567
  Session 03: 1.2059

Object-House dissimilarity trajectory:
  Session 01: 1.8621
  Session 02: 1.4962
  Session 03: 1.5631


In [33]:
# Check if images are actually aligned
subject_id = 'sub-004'
roi_file = BASE_DIR / subject_id / 'ses-01' / 'ROIs' / 'l_face_word_dual_cluster.nii.gz'

for session in ['01', '02', '03']:
    cope_file = BASE_DIR / subject_id / f'ses-{session}' / 'derivatives' / 'fsl' / 'loc' / 'HighLevel.gfeat' / 'cope6.feat' / 'stats' / 'cope1.nii.gz'
    
    roi_img = nib.load(roi_file)
    cope_img = nib.load(cope_file)
    
    print(f"Session {session}:")
    print(f"  Shapes match: {roi_img.shape == cope_img.shape}")
    print(f"  Affines match: {np.allclose(roi_img.affine, cope_img.affine)}")# Check if images are actually aligned
subject_id = 'sub-004'
roi_file = BASE_DIR / subject_id / 'ses-01' / 'ROIs' / 'l_face_word_dual_cluster.nii.gz'

for session in ['01', '02', '03']:
    cope_file = BASE_DIR / subject_id / f'ses-{session}' / 'derivatives' / 'fsl' / 'loc' / 'HighLevel.gfeat' / 'cope6.feat' / 'stats' / 'cope1.nii.gz'
    
    roi_img = nib.load(roi_file)
    cope_img = nib.load(cope_file)
    
    print(f"Session {session}:")
    print(f"  Shapes match: {roi_img.shape == cope_img.shape}")
    print(f"  Affines match: {np.allclose(roi_img.affine, cope_img.affine)}")

Session 01:
  Shapes match: True
  Affines match: True
Session 02:
  Shapes match: True
  Affines match: False
Session 03:
  Shapes match: True
  Affines match: False
Session 01:
  Shapes match: True
  Affines match: True
Session 02:
  Shapes match: True
  Affines match: False
Session 03:
  Shapes match: True
  Affines match: False
